<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill for SOLANA bitcoin</span>


## 🗒️ The tasks of this script
1. Download historical prices for SOLANA as a CSV file
2. Update the path of the CSV file in this notebook to point to the one that you downloaded
5. Create an account on www.hopsworks.ai and get your HOPSWORKS_API_KEY
6. Run notebook to upload the feature on a hopsworks feature storage



### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from utils import *
import json
import re
import os
import warnings
import dotenv
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

/opt/anaconda3/envs/crypto-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### IF YOU WANT TO WIPE OUT ALL OF YOUR FEATURES AND MODELS, run the cell below

In [14]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# with open('../../data/hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
# #proj = hopsworks.login()
#util.purge_project(proj)

---

## Hopsworks API Key
You need to have registered an account on app.hopsworks.ai.
You will be prompted to enter your API key here, unless you set it as the environment variable HOPSWORKS_API_KEY (my preffered approach).

In [2]:
os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
    
project = hopsworks.login()

2024-12-12 14:44:38,029 INFO: Initializing external client
2024-12-12 14:44:38,030 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-12 14:44:40,242 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164448


### Add historical data to hopsworks feature storage

In [15]:
hist_data = pd.read_csv("data/SOL_USD Binance Historical Data.csv")
hist_data.columns = hist_data.columns.str.lower()
hist_data.describe()

,price,open,high,low
count,31.000000,31.000000,31.000000,31.000000
mean,233.173548,232.943871,240.952903,224.819677
std,12.866262,13.037690,12.429703,13.874963
min,209.500000,209.600000,220.170000,201.000000
25%,226.575000,224.005000,235.310000,212.895000
50%,236.200000,235.870000,241.910000,227.670000
75%,237.970000,238.030000,246.805000,233.895000
max,256.570000,256.900000,264.200000,252.750000


In [16]:
hist_data.columns

Index(['date', 'price', 'open', 'high', 'low', 'vol.', 'change %'], dtype='object')

In [ ]:
fs = project.get_feature_store() 
solana_fg = fs.get_or_create_feature_group(
    name='solana',
    description='Solana price',
    version=2,
    primary_key=["date"])


In [19]:
solana_fg.insert(hist_data)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1164448/featurestores/1155151/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270040,"usrMsg":", the provided feature name vol. is invalid. Feature names can only contain lower case characters, numbers and underscores, have to start with a letter and cannot be longer than 63 characters or empty.","errorMsg":"Illegal feature name"}', error code: 270040, error msg: Illegal feature name, user msg: , the provided feature name vol. is invalid. Feature names can only contain lower case characters, numbers and underscores, have to start with a letter and cannot be longer than 63 characters or empty.

### Request solana price from last available data till now

### Getting missing data

In [3]:
url = "https://min-api.cryptocompare.com/data/v2/histoday"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


# Set the parameters for the API request
params = {
    "fsym": "SOL",  # From symbol (e.g., Bitcoin)
    "tsym": "USD",  # To symbol (e.g., US Dollar)
    "limit": 30,    # Number of days of data to retrieve
    "toTs": None,   # Timestamp for the end of the period (None for latest data)
    "api_key": crypto_api_key
}

response = trigger_request(url, params)
response.keys()

dict_keys(['Response', 'Message', 'HasWarning', 'Type', 'RateLimit', 'Data'])

In [6]:
response["Data"]

{'Aggregated': False,
 'TimeFrom': 1731369600,
 'TimeTo': 1733961600,
 'Data': [{'time': 1731369600,
   'high': 225.3,
   'low': 205.03,
   'open': 222.42,
   'volumefrom': 3207955.53,
   'volumeto': 687618053.08,
   'close': 211.86,
   'conversionType': 'direct',
   'conversionSymbol': ''},
  {'time': 1731456000,
   'high': 220.78,
   'low': 201.12,
   'open': 211.86,
   'volumefrom': 3280352.4,
   'volumeto': 695358114.93,
   'close': 215.45,
   'conversionType': 'direct',
   'conversionSymbol': ''},
  {'time': 1731542400,
   'high': 222.79,
   'low': 206.84,
   'open': 215.45,
   'volumefrom': 2390257.3,
   'volumeto': 514053398.69,
   'close': 209.54,
   'conversionType': 'direct',
   'conversionSymbol': ''},
  {'time': 1731628800,
   'high': 220.4,
   'low': 204.62,
   'open': 209.54,
   'volumefrom': 2089124.27,
   'volumeto': 443402898.19,
   'close': 218.51,
   'conversionType': 'direct',
   'conversionSymbol': ''},
  {'time': 1731715200,
   'high': 221.99,
   'low': 213.61,
  

## <span style='color:#ff5f27'> 🌍 STEP 5: Read your CSV file into a DataFrame </span>

The cell below will read up historical air quality data as a CSV file into a Pandas DataFrame

In [17]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

FileNotFoundError: [Errno 2] No such file or directory: '../../data/aristotelous-air-quality.csv'

## <span style='color:#ff5f27'> 🌍 STEP 6: Data cleaning</span>


### Rename columns if needed and drop unneccessary columns

We want to have a DataFrame with 2 columns - `date` and `pm25` after this cell below:

## Check the data types for the columns in your DataFrame

 * `date` should be of type   datetime64[ns] 
 * `pm25` should be of type float64

In [ ]:
# These commands will succeed if your CSV file didn't have a `median` or `timestamp` column
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')
df_aq = util.create_lagged_pm25(df_aq)

df_aq

,date,pm25,lagged_pm25
0,2019-11-07,374.0,999.000000
1,2019-11-08,56.0,686.500000
2,2019-11-09,62.0,476.333333
3,2019-11-10,45.0,164.000000
4,2019-11-11,57.0,54.333333
...,...,...,...
1762,2024-11-03,70.0,108.333333
1763,2024-11-04,47.0,63.666667
1764,2024-11-05,54.0,60.666667
1765,2024-11-06,43.0,57.000000


In [ ]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767 entries, 0 to 1766
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1767 non-null   datetime64[ns]
 1   pm25         1767 non-null   float32       
 2   lagged_pm25  1767 non-null   float64       
dtypes: datetime64[ns](1), float32(1), float64(1)
memory usage: 34.6 KB


## <span style='color:#ff5f27'> 🌍 STEP 7: Drop any rows with missing data </span>
It will make the model training easier if there is no missing data in the rows, so we drop any rows with missing data.

In [ ]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25,lagged_pm25
0,2019-11-07,374.0,999.000000
1,2019-11-08,56.0,686.500000
2,2019-11-09,62.0,476.333333
3,2019-11-10,45.0,164.000000
4,2019-11-11,57.0,54.333333
...,...,...,...
1762,2024-11-03,70.0,108.333333
1763,2024-11-04,47.0,63.666667
1764,2024-11-05,54.0,60.666667
1765,2024-11-06,43.0,57.000000


## <span style='color:#ff5f27'> 🌍 STEP 8: Add country, city, street, url to the DataFrame </span>

Your CSV file may have many other air quality measurement columns. We will only work with the `pm25` column.

We add the columns for the country, city, and street names that you changed for your Air Quality sensor.

We also want to make sure the `pm25` column is a float32 data type.

In [ ]:
# Your sensor may have columns we won't use, so only keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url
df_aq

,date,pm25,lagged_pm25,country,city,street,url
0,2019-11-07,374.0,999.000000,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
1,2019-11-08,56.0,686.500000,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
2,2019-11-09,62.0,476.333333,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
3,2019-11-10,45.0,164.000000,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
4,2019-11-11,57.0,54.333333,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
...,...,...,...,...,...,...,...
1762,2024-11-03,70.0,108.333333,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
1763,2024-11-04,47.0,63.666667,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
1764,2024-11-05,54.0,60.666667,greece,athens,aristotelous,https://api.waqi.info/feed/@12414
1765,2024-11-06,43.0,57.000000,greece,athens,aristotelous,https://api.waqi.info/feed/@12414


---

## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

## <span style='color:#ff5f27'> 🌍 STEP 9: Download the Historical Weather Data </span>

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [ ]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

Coordinates 33.98945236206055°N -83.37078857421875°E
Elevation 220.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [ ]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1837 entries, 0 to 1836
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         1837 non-null   datetime64[ns]
 1   temperature_2m_mean          1837 non-null   float32       
 2   precipitation_sum            1837 non-null   float32       
 3   wind_speed_10m_max           1837 non-null   float32       
 4   wind_direction_10m_dominant  1837 non-null   float32       
 5   city                         1837 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 71.8+ KB


## <span style='color:#ff5f27'> 🌍 STEP 10: Define Data Validation Rules </span>

We will validate the air quality measurements (`pm25` values) before we write them to Hopsworks.

We define a data validation rule (an expectation in Great Expectations) that ensures that `pm25` values are not negative or above the max value available by the sensor.

We will attach this expectation to the air quality feature group, so that we validate the `pm25` data every time we write a DataFrame to the feature group. We want to prevent garbage-in, garbage-out.

In [ ]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

## Expectations for Weather Data
Here, we define an expectation for 2 columns in our weather DataFrame - `precipitation_sum` and `wind_speed_10m_max`, where we expect both values to be greater than zero, but less than 1000.

In [ ]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

---

### <span style="color:#ff5f27;"> 🔮 STEP 11: Connect to Hopsworks and save the sensor country, city, street names as a secret</span>

In [ ]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [ ]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "greece", "city": "athens", "street": "aristotelous", "aqicn_url": "https://api.waqi.info/feed/@12414", "latitude": 33.96, "longitude": -83.38}


### <span style="color:#ff5f27;"> 🔮 STEP 12: Create the Feature Groups and insert the DataFrames in them </span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each air quality sensor measurement is uniquely identified by `country`, `street`, and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality_with_lagged',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

#### Insert the DataFrame into the Feature Group

In [ ]:
air_quality_fg.insert(df_aq)

2024-11-18 22:42:38,590 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164448/fs/1155151/fg/1348979


Uploading Dataframe: 0.00% |          | Rows 0/1767 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_with_lagged_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1164448/jobs/named/air_quality_with_lagged_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x16ae83310>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 672806
         }
       },
       "result": {
         "observed_value": 18.0,
         "element_count": 1767,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T09:42:38.000590Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "uns

#### Enter a description for each feature in the Feature Group

In [ ]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")
air_quality_fg.update_feature_description("lagged_pm25", "Three day average of pm25")


### <span style='color:#ff5f27'> 🌦 Weather Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each weather measurement is uniquely identified by `city` and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [ ]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

#### Insert the DataFrame into the Feature Group

In [ ]:
# Insert data
weather_fg.insert(weather_df)

2024-11-18 22:42:51,618 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164448/fs/1155151/fg/1339763


Uploading Dataframe: 0.00% |          | Rows 0/1837 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1164448/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x16ae81710>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 663568
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 1837,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T09:42:51.000618Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwe

#### Enter a description for each feature in the Feature Group

In [ ]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Daily Feature Pipeline 
 </span> 


## <span style="color:#ff5f27;">⏭️ **Exercises:** 
 </span> 

Extra Homework:

  * Try adding a new feature based on a rolling window of 3 days for 'pm25'
      * This is not easy, as forecasting more than 1 day in the future, you won't have the previous 3 days of pm25 measurements.
      * df.set_index("date").rolling(3).mean() is only the start....
  * Parameterize the notebook, so that you can provide the `country`/`street`/`city`/`url`/`csv_file` as parameters. 
      * Hint: this will also require making the secret name (`SENSOR_LOCATION_JSON`), e.g., add the street name as part of the secret name. Then you have to pass that secret name as a parameter when running the operational feature pipeline and batch inference pipelines.
      * After you have done this, collect the street/city/url/csv files for all the sensors in your city or region and you make dashboards for all of the air quality sensors in your city/region. You could even then add a dashboard for your city/region, as done [here for Poland](https://github.com/erno98/ID2223).

Improve this AI System
  * As of mid 2024, there is no API call available to download historical data from the AQIN website. You could improve this system by writing a PR to download the CSV file using Python Selenium and the URL for the sensor.


---